# **SENTIMENTAL ANALYSIS:**

**LOADING THE DATA:**

In [1]:
#LOADING THE DATASET
import pandas as pd 
df = pd.read_json('Arts_Crafts_and_Sewing_5.json',lines=True)
df.sample(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
338,5,False,"01 9, 2018",A1B1PFNTX9U9ME,194757504X,{'Color:': ' 9 X 12 - 2 Pack'},L Moody,"high quality, thick paper. markers did not ble...","high quality, thick paper",1515456000,2,NaN
150417,5,False,"04 7, 2011",A2E3NAL9ECE7VS,B001IKES5O,{'Size:': ' 16 Oz'},H. Erickson,I am a paper crafter (not scrapbooker) who use...,How to use the matte finish,1302134400,375,NaN
359858,5,True,"04 9, 2017",AD0L3R25NWEPX,B00QDLD3YK,NaN,Ed,"Excellent product, installs easily, and extrem...",Five Stars,1491696000,NaN,NaN
8769,5,True,"02 1, 2015",A1U3Q1EMEF1H76,B0007D63NI,NaN,mary mendez,Purchased for my 10 year old artist. She loves...,Good deal,1422748800,NaN,NaN
209525,5,True,"11 24, 2017",A2IFKMKKXV8AVU,B004BSDLS0,"{'Size:': ' 2 Count', 'Package Type:': ' Stand...",Dan Dreifort,Remember when people used to buy nice scissors...,Cheap. Cheap. Good. Disposable. Welcome.,1511481600,8,NaN
458903,5,True,"09 13, 2016",AQW2DS6BSL70H,B00BCIWRLO,NaN,mamadee,How did I live without this brush? Nuff said....,Buy it if you're looking for a good hake.,1473724800,NaN,NaN
154293,5,True,"12 30, 2013",A2634HEIL9C6CD,B000WE04KU,"{'Color:': ' 10""x100yd Roll Tear-away'}",Linda L. Thompson,I love the fact that this roll will last me fo...,Embroidery Stabilizer,1388361600,NaN,NaN
53347,5,True,"04 28, 2015",AQDFPSIR1RZE6,B000Y3IVUE,{'Size:': ' 1 PACK'},Deenie,Nice sized needles. They are plastic; howeve...,Great little needles!,1430179200,NaN,NaN
432736,5,True,"02 11, 2017",A3RFCS5RA3PVF6,B001UJYBKA,NaN,Slnakai,These stamps are fantastic. They have an amusi...,Great assortment of Birds & Bugs,1486771200,NaN,NaN
333802,5,True,"10 10, 2015",A1C4GDSSMSKE8W,B00JLX6HCW,NaN,Marla Allison,awesome mini's,love mini's,1444435200,NaN,NaN


**GET THE BING LIU LEXICON:**

In [2]:
#Downloading the opinion lexicon
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/hepsiba/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


**CREATING THE 'DICTIONARY':**

In [3]:
# Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}

# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
     word_dict[word] = pos_score

# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
     word_dict[word] = neg_score

[nltk_data] Downloading package punkt to /Users/hepsiba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**FUNCTION TO DO THE MATH:**

In [4]:
#function to do the math
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
            return sentiment_score

**CALLING THE FUNCTION:**

In [5]:
#calling the function
df['text'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

In [6]:
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1.0
1,5,I'm a fairly experienced knitter of the one-co...,1.0
2,4,Great book but the index is terrible. Had to w...,1.0
3,5,I purchased the Kindle edition which is incred...,1.0
4,5,Very well laid out and very easy to read.\n\nT...,1.0
5,5,"Beginning her career as a freelance knitter, M...",1.0
6,5,This is a terrific stitch handbook (and I have...,1.0
7,4,The book needs to be coil bound. The content i...,1.0
8,5,I really am enjoying this book! I like the siz...,1.0
9,5,Just received this book and looked over it cov...,1.0


**SENTIMENT COUNT:**

In [7]:
#Overall rating
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.139887
2,0.174485
3,0.458573
4,0.770115
5,0.888414
